In [1]:
pip install dask[dataframe]

In [4]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [5]:
#Load the data using pandas read_csv function. This will generate a data frame we will use through the work.
cleaned_goalkeeeprs_df = pd.read_csv('goalkeepers_rating.csv')
cleaned_midfielders_df = pd.read_csv('midfielders_rating.csv')
cleaned_forwarders_df = pd.read_csv('forwarders_rating.csv')
cleaned_defenders_df = pd.read_csv('defenders_rating.csv')

In [6]:
#Show the first and last few records to check data is loading correctly
print(cleaned_goalkeeeprs_df.head())
print(cleaned_goalkeeeprs_df.tail())

         date  original_rating  goals  assists  shots_ontarget  \
0  2016-06-10             6.10      0        0               0   
1  2016-06-10             6.20      0        0               0   
2  2016-06-11             6.92      0        0               0   
3  2016-06-11             5.99      0        0               0   
4  2016-06-11             6.33      0        0               0   

   shots_offtarget  shotsblocked  chances2score  drib_success  drib_unsuccess  \
0                0             0              0             0               0   
1                0             0              0             0               0   
2                0             0              0             0               0   
3                0             0              0             0               0   
4                0             0              0             0               0   

   ...  missed_penalties  owngoals  degree_centrality  betweenness_centrality  \
0  ...                 0         0 

In [7]:
import pandas as pd

df = cleaned_goalkeeeprs_df

# Iterate through each column and check for columns with a single unique value
single_value_columns = [column for column in df.columns if df[column].nunique() == 1]

# Print columns with a single unique value
print("Columns with a single unique value:")
for column in single_value_columns:
    print(column)

Columns with a single unique value:
goals
shots_ontarget
rcards
offsides
missed_penalties
flow_success
minutesPlayed


In [8]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['goals',
'shots_ontarget',
'rcards',
'offsides',
'missed_penalties',
'flow_success',
'minutesPlayed']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_goalkeeeprs_df = cleaned_goalkeeeprs_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_goalkeeeprs_df.head()

,date,original_rating,assists,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,...,saves_itb,saves_otb,saved_pen,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration
0,2016-06-10,6.10,0,0,0,0,0,0,0,28,...,2,3,0,0,0.785714,0.076854,0.574830,0.130435,0.0,90
1,2016-06-10,6.20,0,0,0,0,0,0,0,15,...,1,0,0,0,0.642857,0.041239,0.548701,0.109589,0.0,90
2,2016-06-11,6.92,0,0,0,0,0,0,0,32,...,4,1,0,0,0.615385,0.032686,0.527473,0.171875,0.0,90
3,2016-06-11,5.99,0,0,0,0,0,0,0,32,...,2,0,0,0,1.153846,0.084425,0.651584,0.289474,0.0,90
4,2016-06-11,6.33,0,0,0,0,0,0,0,26,...,1,1,0,0,1.076923,0.128150,0.615385,0.157143,0.0,90


In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [10]:
cleaned_goalkeeeprs_df.head()

,date,original_rating,assists,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,...,saves_itb,saves_otb,saved_pen,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration
0,2016-06-10,6.10,0,0,0,0,0,0,0,28,...,2,3,0,0,0.785714,0.076854,0.574830,0.130435,0.0,90
1,2016-06-10,6.20,0,0,0,0,0,0,0,15,...,1,0,0,0,0.642857,0.041239,0.548701,0.109589,0.0,90
2,2016-06-11,6.92,0,0,0,0,0,0,0,32,...,4,1,0,0,0.615385,0.032686,0.527473,0.171875,0.0,90
3,2016-06-11,5.99,0,0,0,0,0,0,0,32,...,2,0,0,0,1.153846,0.084425,0.651584,0.289474,0.0,90
4,2016-06-11,6.33,0,0,0,0,0,0,0,26,...,1,1,0,0,1.076923,0.128150,0.615385,0.157143,0.0,90


In [11]:
cleaned_goalkeeeprs_df.describe()

,original_rating,assists,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,passes_acc,...,saves_itb,saves_otb,saved_pen,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration
count,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,...,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000,1571.000000
mean,6.665111,0.001273,0.002546,0.000637,0.003183,0.004456,0.001910,0.005729,29.639083,16.187142,...,1.609803,1.496499,0.026735,0.004456,0.801321,0.086446,0.560934,0.221086,0.001316,90.305538
std,0.770222,0.035669,0.050411,0.025230,0.056343,0.066624,0.043671,0.075496,8.890930,6.783315,...,1.410111,1.354570,0.161358,0.066624,0.193114,0.051721,0.056192,0.069660,0.012036,3.013065
min,3.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.344898,0.040816,0.000000,90.000000
25%,6.155000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,11.000000,...,1.000000,0.000000,0.000000,0.000000,0.666667,0.051194,0.522667,0.173333,0.000000,90.000000
50%,6.660000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.000000,15.000000,...,1.000000,1.000000,0.000000,0.000000,0.800000,0.078041,0.568116,0.217391,0.000000,90.000000
75%,7.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.000000,20.000000,...,2.000000,2.000000,0.000000,0.000000,0.933333,0.110773,0.593939,0.262500,0.000000,90.000000
max,9.260000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,77.000000,57.000000,...,8.000000,8.000000,1.000000,1.000000,1.357143,0.363904,0.768627,0.504673,0.200000,120.000000


In [12]:
# lets move towards splitting the data.

# get the locations
y = cleaned_goalkeeeprs_df['original_rating']
X = cleaned_goalkeeeprs_df.drop(columns=['original_rating'], axis=1)

In [13]:
X.head()

,date,assists,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,passes_acc,...,saves_itb,saves_otb,saved_pen,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration
0,2016-06-10,0,0,0,0,0,0,0,28,12,...,2,3,0,0,0.785714,0.076854,0.574830,0.130435,0.0,90
1,2016-06-10,0,0,0,0,0,0,0,15,11,...,1,0,0,0,0.642857,0.041239,0.548701,0.109589,0.0,90
2,2016-06-11,0,0,0,0,0,0,0,32,14,...,4,1,0,0,0.615385,0.032686,0.527473,0.171875,0.0,90
3,2016-06-11,0,0,0,0,0,0,0,32,17,...,2,0,0,0,1.153846,0.084425,0.651584,0.289474,0.0,90
4,2016-06-11,0,0,0,0,0,0,0,26,13,...,1,1,0,0,1.076923,0.128150,0.615385,0.157143,0.0,90


In [14]:
y.head()

0    6.10
1    6.20
2    6.92
3    5.99
4    6.33
Name: original_rating, dtype: float64

In [15]:
# Create DataFrame
# Convert 'date' column to datetime with error handling
X['date'] = pd.to_datetime(X['date'], errors='coerce')

# Check for any NaT values
if X['date'].isna().any():
    print("Some dates couldn't be parsed. Handling them as NaT (Not a Time).")

# Extract year, month, and day
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day

# Drop the original 'date' column
X = X.drop(columns=['date'])

# Display the DataFrame
print(X.head())

   assists  shots_offtarget  shotsblocked  chances2score  drib_success  \
0        0                0             0              0             0   
1        0                0             0              0             0   
2        0                0             0              0             0   
3        0                0             0              0             0   
4        0                0             0              0             0   

   drib_unsuccess  keypasses  touches  passes_acc  passes_inacc  ...  \
0               0          0       28          12             6  ...   
1               0          0       15          11             2  ...   
2               0          0       32          14             7  ...   
3               0          0       32          17             4  ...   
4               0          0       26          13             6  ...   

   owngoals  degree_centrality  betweenness_centrality  closeness_centrality  \
0         0           0.785714            

In [16]:
X.columns

Index(['assists', 'shots_offtarget', 'shotsblocked', 'chances2score',
       'drib_success', 'drib_unsuccess', 'keypasses', 'touches', 'passes_acc',
       'passes_inacc', 'crosses_acc', 'crosses_inacc', 'lballs_acc',
       'lballs_inacc', 'grduels_w', 'grduels_l', 'aerials_w', 'aerials_l',
       'poss_lost', 'fouls', 'wasfouled', 'clearances', 'stop_shots',
       'interceptions', 'tackles', 'dribbled_past', 'tballs_acc',
       'tballs_inacc', 'ycards', 'dangmistakes', 'countattack', 'goals_ag_otb',
       'goals_ag_itb', 'saves_itb', 'saves_otb', 'saved_pen', 'owngoals',
       'degree_centrality', 'betweenness_centrality', 'closeness_centrality',
       'flow_centrality', 'betweenness2goals', 'game_duration', 'year',
       'month', 'day'],
      dtype='object')

In [17]:
X.dtypes

assists                     int64
shots_offtarget             int64
shotsblocked                int64
chances2score               int64
drib_success                int64
drib_unsuccess              int64
keypasses                   int64
touches                     int64
passes_acc                  int64
passes_inacc                int64
crosses_acc                 int64
crosses_inacc               int64
lballs_acc                  int64
lballs_inacc                int64
grduels_w                   int64
grduels_l                   int64
aerials_w                   int64
aerials_l                   int64
poss_lost                   int64
fouls                       int64
wasfouled                   int64
clearances                  int64
stop_shots                  int64
interceptions               int64
tackles                     int64
dribbled_past               int64
tballs_acc                  int64
tballs_inacc                int64
ycards                      int64
dangmistakes  

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Normalize numerical columns using Robust Scaler
numerical_columns = X.select_dtypes(include=['int64', 'float64','int32']).columns
scaler = MinMaxScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Display the DataFrame
print(X.head())

   assists  shots_offtarget  shotsblocked  chances2score  drib_success  \
0      0.0              0.0           0.0            0.0           0.0   
1      0.0              0.0           0.0            0.0           0.0   
2      0.0              0.0           0.0            0.0           0.0   
3      0.0              0.0           0.0            0.0           0.0   
4      0.0              0.0           0.0            0.0           0.0   

   drib_unsuccess  keypasses   touches  passes_acc  passes_inacc  ...  \
0             0.0        0.0  0.300000    0.181818      0.352941  ...   
1             0.0        0.0  0.114286    0.163636      0.117647  ...   
2             0.0        0.0  0.357143    0.218182      0.411765  ...   
3             0.0        0.0  0.357143    0.272727      0.235294  ...   
4             0.0        0.0  0.271429    0.200000      0.352941  ...   

   owngoals  degree_centrality  betweenness_centrality  closeness_centrality  \
0       0.0           0.529412      

In [19]:
X.shape

(1571, 46)

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, split the train+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Display the sizes of the splits to ensure they are correct
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 942
Validation set size: 314
Test set size: 315


In [21]:
X_train.head()

,assists,shots_offtarget,shotsblocked,chances2score,drib_success,drib_unsuccess,keypasses,touches,passes_acc,passes_inacc,...,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,year,month,day
808,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.485714,0.236364,0.411765,...,0.0,0.588235,0.174433,0.685438,0.706263,0.0,0.0,0.5,1.000000,0.966667
511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257143,0.218182,0.176471,...,0.0,0.431373,0.298828,0.470929,0.292449,0.0,0.0,0.5,0.909091,0.133333
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.371429,0.272727,0.294118,...,0.0,0.760784,0.315618,0.587737,0.397829,0.0,0.0,0.5,1.000000,0.266667
1440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.185714,0.163636,0.058824,...,0.0,0.541176,0.178584,0.470929,0.259723,0.0,0.0,1.0,0.363636,0.400000
190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.614286,0.400000,0.470588,...,0.0,0.705882,0.251837,0.685438,0.902527,0.0,0.0,0.5,0.636364,0.866667


In [32]:
# Save each DataFrame as a CSV file
X.to_csv('goalkeepers_rating_X.csv', index=False)
y.to_csv('goalkeepers_rating_y.csv', index=False)


print("DataFrames have been saved as CSV files.")

DataFrames have been saved as CSV files.
